<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-assets/phoenix/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Tracing and Evaluating a LangChain Application</h1>

LangChain provides high-level APIs that enable users to build powerful applications in a few lines of code. However, it can be challenging to understand what is going on under the hood and to pinpoint the cause of issues. Phoenix makes your LLM applications *observable* by visualizing the underlying structure of each call to your query engine and surfacing problematic "spans" of execution based on latency, token count, or other evaluation metrics.

In this tutorial, you will:
- Build a simple question and answer app using LangChain that uses retrieval-augmented generation to answer questions over the Arize documentation,
- Record trace data in OpenInference format,
- Inspect the traces and spans of your application to identify sources of latency and cost,
- Export your trace data as a pandas dataframe and run an LLM-assisted evaluation to measure the precision@k of your retrieval step.

ℹ️ This notebook requires an OpenAI API key.

## 1. Install Dependencies and Import Libraries

Install Phoenix, LangChain, and OpenAI.

In [ ]:
!pip install "langchain>=0.0.332" "openai>1" arize-phoenix tiktoken

Import libraries.

In [ ]:
import json
import os
from getpass import getpass
from urllib.request import urlopen

import numpy as np
import pandas as pd
import phoenix as px
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import KNNRetriever
from phoenix.trace.langchain import LangChainInstrumentor, OpenInferenceTracer
from tqdm import tqdm

## 2. Launch Phoenix

You can run Phoenix in the background to collect trace data emitted by any LangChain application that has been instrumented with the `OpenInferenceTracer`.

Launch Phoenix and follow the instructions in the cell output to open the Phoenix UI (the UI should be empty because we have yet to run a LangChain application).

In [ ]:
px.launch_app()

## 3. Configure Your OpenAI API Key

Set your OpenAI API key if it is not already set as an environment variable.

In [ ]:
if os.environ.get("OPENAI_API_KEY") is None:
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
    os.environ["OPENAI_API_KEY"] = openai_api_key

### 4. Build Your LangChain Application

This example uses a `RetrievalQA` chain over a pre-built index of the Arize documentation, but you can use whatever LangChain application you like.

Download your pre-built index from cloud storage and instantiate your storage context.

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
df = pd.read_parquet(
    "http://storage.googleapis.com/arize-assets/phoenix/datasets/"
    "unstructured/llm/context-retrieval/langchain-pinecone/database.parquet"
)
knn_retriever = KNNRetriever(
    index=np.stack(df["text_vector"]),
    texts=df["text"].tolist(),
    embeddings=OpenAIEmbeddings(),
)
chain_type = "stuff"  # stuff, refine, map_reduce, and map_rerank
chat_model_name = "gpt-3.5-turbo"
llm = ChatOpenAI(model_name=chat_model_name)
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type=chain_type,
    retriever=knn_retriever,
)

Instantiate an `OpenInferenceTracer` to store your data in [OpenInference format](https://github.com/Arize-ai/open-inference-spec). OpenInference is an open standard for capturing and storing LLM application traces and enables production LLMapp servers to seamlessly integrate with LLM observability solutions such as Phoenix. Note that we are using the `LangChainInstrumentor` to add the tracer to every chain in our application but you can also add the tracer to individual chains. For the full details on the `OpenInferenceTracer` and `LangChainInstrumentor`, see the [integrations documentation](https://docs.arize.com/phoenix/langchain).

In [ ]:
tracer = OpenInferenceTracer()
LangChainInstrumentor(tracer=tracer).instrument()

## 5. Run Your Query Engine and View Your Traces in Phoenix

Download a sample of queries commonly asked of the Arize documentation.

In [ ]:
url = "http://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/llm/context-retrieval/arize_docs_queries.jsonl"
queries = []
with urlopen(url) as response:
    for line in response:
        line = line.decode("utf-8").strip()
        data = json.loads(line)
        queries.append(data["query"])
queries[:10]

Run a few queries.

In [ ]:
for query in tqdm(queries[:10]):
    chain.run(query)

Check the Phoenix UI as your queries run. Your traces should appear in real time.